# Importy

In [1]:
import os
import numpy as np
import pandas as pd 
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 # Kształty, kontury

# Do entropii
from skimage.color import rgb2gray
from skimage.measure import shannon_entropy

# Do ekstrakcji cech
from skimage.feature import graycomatrix, graycoprops
from skimage import util
from scipy import ndimage as ndi

# Do embedingu
from transformers import CLIPModel, CLIPProcessor
import torch

# Rzutowanie
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Klasteryzacja
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN


2025-05-23 19:51:25.589126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748029885.828904      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748029885.906920      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Wczytywanie obrazów do listy i sprawdzanie poprawności

In [ ]:
def validate_images(root_dir, expected_shape=(None, None, 3)):

    valid_images = []
    corrupted_files = []
    invalid_shapes = []
    
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
                
            path = os.path.join(subdir, file)
            try:
                with Image.open(path) as img:
                    img = img.convert("RGB")
                    img_array = np.array(img)

                if expected_shape[0] is not None and img_array.shape[0] != expected_shape[0]:
                    raise ValueError(f"Nieprawidłowa wysokość: {img_array.shape[0]} (oczekiwano {expected_shape[0]})")
                if expected_shape[1] is not None and img_array.shape[1] != expected_shape[1]:
                    raise ValueError(f"Nieprawidłowa szerokość: {img_array.shape[1]} (oczekiwano {expected_shape[1]})")
                if expected_shape[2] is not None and img_array.shape[2] != expected_shape[2]:
                    raise ValueError(f"Nieprawidłowa liczba kanałów: {img_array.shape[2]} (oczekiwano {expected_shape[2]})")
                
                valid_images.append(img_array)
                
            except (UnidentifiedImageError, ValueError, OSError) as e:
                corrupted_files.append(path)
                print(f"Uszkodzony/nieprawidłowy plik: {path} | Błąd: {str(e)}")
            except Exception as e:
                corrupted_files.append(path)
                print(f"Nieoczekiwany błąd w pliku {path}: {str(e)}")
    

    print(f"\nWynik walidacji:")
    print(f"Poprawne obrazy: {len(valid_images)}")
    print(f"Uszkodzone pliki: {len(corrupted_files)}")
    print(f"Obrazy z nieprawidłowym kształtem: {len(invalid_shapes)}")
    
    return valid_images, corrupted_files, invalid_shapes


root_dir = "/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset"
expected_shape = (600, 600, 3)

images, corrupted_files, invalid_shapes = validate_images(root_dir, expected_shape)


In [ ]:
images[0].shape

In [ ]:
#Do sprawdzenia, jak dobrze nam podzieliło
arr = np.concatenate([np.zeros(1232), np.ones(916)])
pistacje_df = pd.DataFrame({'pistacja': arr})

### Wyświetlenie obrazów

In [ ]:
# Wyświetlenie 3 pierwszych obrazów
plt.figure(figsize=(12, 4))

for i in range(3):
    plt.subplot(1, 3, i + 1)  
    plt.imshow(images[i])     
    plt.axis('off')          
    plt.title(f"Obraz {i + 1}")

plt.tight_layout()
plt.show()

# EDA

In [ ]:
def cechy_ksztaltu(img_array):
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    
    _, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        # Pole powierzchni (w pikselach)
        area = cv2.contourArea(contour)
        # Obwód obiektu
        perimeter = cv2.arcLength(contour, True)
        # Prostokąt ograniczający (szerokość i wysokość)
        x, y, width, height = cv2.boundingRect(contour)
        circularity = 4 * np.pi * area / (perimeter ** 2)
        aspect_ratio = float(width) / height
    else:
        area = 0
        perimeter = 0
        width = 0
        height = 0
        circularity = 0
        aspect_ratio = 0

    # Zwracamy słownik z cechami
    return {
        "area": area,
        "perimeter": perimeter,
        "width": width,
        "height": height,
        "circularity": circularity,
        "aspect_ratio": aspect_ratio
    }



In [ ]:
features = []

for i, img_array in enumerate(images):
    # Konwersja do skali szarości
    gray = np.mean(img_array, axis=2)  # Średnia z RGB kanałów

    avg_gray = np.mean(gray)
    contrast = np.max(gray) - np.min(gray)

    # HSV: nasycenie i jasność
    hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    s_mean = np.mean(hsv[:, :, 1])  # Saturacja
    v_mean = np.mean(hsv[:, :, 2])  # Jasność


    ksztalt = cechy_ksztaltu(img_array)
    features.append({
        "Obraz": f"Obraz {i+1}",
        "Średni odcień szarości": round(avg_gray, 2),
        "Kontrast": round(contrast, 2),
        "r_mean": np.mean(img_array[:, :, 0]), 
        "g_mean": np.mean(img_array[:, :, 1]),  
        "b_mean": np.mean(img_array[:, :, 2]),
        "r_std": np.std(img_array[:, :, 0]),
        "g_std": np.std(img_array[:, :, 1]),
        "b_std": np.std(img_array[:, :, 2]),
        "area": ksztalt["area"],
        "perimeter": ksztalt["perimeter"],
        "width": ksztalt["width"],
        "height": ksztalt["height"],
        "aspect_ratio": ksztalt["aspect_ratio"],
        "circularity": ksztalt["circularity"],
        "saturation_mean": round(s_mean, 2),
        "brightness_mean": round(v_mean, 2)
    })


df = pd.DataFrame(features)

print(df)

## Wykresy

In [ ]:
sns.histplot(df["Kontrast"], bins=30, kde=True)
plt.title("Rozkład kontrastu w obrazach")
plt.xlabel("Kontrast")
plt.ylabel("Liczba obrazów")
plt.show()

In [ ]:
sns.boxplot(data=df[["r_mean", "g_mean", "b_mean"]], palette=["red", "green", "blue"])
plt.title("Rozkład średnich wartości RGB")
plt.ylabel("Średnia wartość kanału")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Histogramy z kernel density estimate (KDE)
sns.histplot(df["r_mean"], color='red', kde=True, label='R (Red)', stat='density', bins=30, alpha=0.25)
sns.histplot(df["g_mean"], color='green', kde=True, label='G (Green)', stat='density', bins=30, alpha=0.25)
sns.histplot(df["b_mean"], color='blue', kde=True, label='B (Blue)', stat='density', bins=30, alpha=0.25)

plt.title("Rozkład średnich wartości kanałów RGB (na obraz)")
plt.xlabel("Średnia wartość kanału (0–255)")
plt.ylabel("Gęstość")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

sns.histplot(df["brightness_mean"], bins=30, kde=True)
plt.title("Rozkład jasności w obrazach")
plt.xlabel("Jasność")
plt.ylabel("Liczba obrazów")
plt.show()

In [ ]:
sns.histplot(df["saturation_mean"], bins=50, kde=True)
plt.title("Rozkład saturacji w obrazach")
plt.xlabel("Saturacja")
plt.ylabel("Liczba obrazów")
plt.show()

In [ ]:
sns.histplot(df["width"], bins=50, label="Szerokość", kde=True)
sns.histplot(df["height"], bins=50, label="Długość", kde=True)
plt.title("Rozkład szerokości i długości pistacji")
plt.xlabel("Wartość w pixelach")
plt.ylabel("Liczba obrazów")
plt.legend()
plt.show()

Szerokość jest lewoskośna, długość prawoskośna.

In [ ]:
sns.histplot(df["aspect_ratio"], bins=50, label="Wydłużenie", kde=True)
sns.histplot(df["circularity"], bins=50, label="Zaokrąglrnie", kde=True)
plt.title("Rozkład wydłużenia i zaokrąglenia pistacji")
plt.xlabel("Wartość w pixelach")
plt.ylabel("Liczba obrazów")
plt.legend()
plt.show()

In [ ]:
sns.histplot(df["perimeter"], bins=50, kde=True)
plt.title("Obwód pistacji")
plt.xlabel("Wartość w pixelach")
plt.ylabel("Liczba obrazów")
plt.show()

In [ ]:
sns.histplot(df["area"], bins=50, kde=True)
plt.title("Pole pistacji")
plt.xlabel("Wartość w pixelach")
plt.ylabel("Liczba obrazów")
plt.show()

# Przetwarzanie obrazów

Dla obrazów: np. zmiana rozmiaru i standaryzacja obrazów, skala szarości, augmentacja danych (odbicie lustrzane, rotacja, przycięcie,
jasność), ekstrakcja cech (Haralick, Gabor), entropia, zanurzenia (CLIP, ViT, ResNet), inne

### Przycięcie obrazów

In [ ]:
# obraz zmniejszony, aby było mniej czarnego tła, nie ucina pistacji, bo wysokość większa od max z height
images_np = np.array(images)

def crop_center(images, crop_size=540):
    h, w = images.shape[1:3]
    start_h = (h - crop_size) // 2
    start_w = (w - crop_size) // 2
    return images[:, start_h:start_h+crop_size, start_w:start_w+crop_size, :]

images_cropped = crop_center(images_np)


In [ ]:
# Wyświetlenie 3 pierwszych obrazów
plt.figure(figsize=(12, 4))

for i in range(3):
    plt.subplot(1, 3, i + 1)  
    plt.imshow(images_cropped[i])     
    plt.axis('off')          
    plt.title(f"Obraz {i + 1}")

plt.tight_layout()
plt.show()

In [ ]:
images_cropped[0].shape

### Obliczenie entropii

- niska entropia - obszary jednorodne, gładkie
- wysoka entropia - obszare bogate w detale i zmienność

- skala entropii w naszym wypadku wynosi od 0 do 8.

In [ ]:
def calculate_entropy(image):
    image = rgb2gray(image)
    return shannon_entropy(image)

entropy_values = [calculate_entropy(img) for img in images_cropped]

df['entropy'] = entropy_values

df.head(5)

## Ekstrakcja cech Haralicka i Gabora

Funkcja do ekstrakcji wybranych cech Harlicka z obrazu, właściwie to z GLCM (macierzy współwystępowania).
Sprawdzane jest 6 cech (properties), związane ze strukturą obiektów na zdjęciu. Ich opis: https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops

In [ ]:
def extract_haralick_features(image):
    # Konwersja do skali szarości
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image
        
    # Normalizacja wartości pikseli do zakresu [0, 255]
    if gray.dtype != np.uint8:
        gray = util.img_as_ubyte(gray)
    
    # Redukcja liczby poziomów szarości do 32 (przyspiesza obliczenia)
    gray = (gray / 8).astype(np.uint8)
    
    # Obliczenie macierzy GLCM dla czterech kierunków i odległości 1
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°
    glcm = graycomatrix(gray, distances, angles, levels=32, symmetric=True, normed=True)
    
    # Obliczenie właściwości GLCM (cech Haralicka)
    features = {}
    properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    
    for prop in properties:
        feature_values = graycoprops(glcm, prop).flatten()
        # Dla każdej właściwości zapisujemy średnią i odchylenie standardowe z czterech kierunków
        features[f'haralick_{prop}_mean'] = np.mean(feature_values)
        features[f'haralick_{prop}_std'] = np.std(feature_values)
    
    return features

Funkcja do stworzenia jądra dla filtra Gabora, można podawać różne wartości wpsółczynników, co pomoże znaleźć odpowiednie wartości.

In [ ]:
def gabor_kernel(sigma, theta, lambd, gamma, psi):
    """
    Tworzy jądro filtru Gabora.
    
    Parametry:
    sigma (float): Odchylenie standardowe funkcji Gaussa
    theta (float): Orientacja filtru w radianach
    lambd (float): Długość fali funkcji sinusoidalnej
    gamma (float): Współczynnik kształtu przestrzennego (stosunek wysokości do szerokości)
    psi (float): Przesunięcie fazowe
    
    Zwraca:
    numpy.ndarray: Jądro filtru Gabora
    """
    # Rozmiar jądra zależy od sigma (większe sigma = większe jądro)
    sigma_x = sigma
    sigma_y = sigma / gamma
    
    # Rozmiar jądra jako wielokrotność sigma
    nstds = 3
    c = np.cos(theta)
    s = np.sin(theta)
    
    # Ustalamy rozmiar jądra
    xmax = max(abs(nstds * sigma_x * c), abs(nstds * sigma_y * s))
    ymax = max(abs(nstds * sigma_x * s), abs(nstds * sigma_y * c))
    xmax = np.ceil(max(1, xmax))
    ymax = np.ceil(max(1, ymax))
    
    # Tworzymy siatkę współrzędnych
    y, x = np.mgrid[-ymax:ymax+1, -xmax:xmax+1]
    
    # Obrót współrzędnych
    x_theta = x * c + y * s
    y_theta = -x * s + y * c
    
    # Oblienie jądra Gabora
    gb = np.exp(-.5 * (x_theta**2 / sigma_x**2 + y_theta**2 / sigma_y**2)) * np.cos(2 * np.pi * x_theta / lambd + psi)
    
    return gb

Funkcja do ekstrakcji cech Gabora z obrazu, związane z kształtem. 

In [ ]:
def extract_gabor_features(image, num_orientations= 4, num_scales=2):
    """
    Ekstrahuje cechy z filtru Gabora dla obrazu.
    
    Parametry:
    image (numpy.ndarray): Obraz wejściowy
    num_orientations (int): Liczba orientacji filtrów
    num_scales (int): Liczba skal filtrów
    
    Zwraca:
    dict: Słownik z cechami Gabora
    """
    # Konwersja do skali szarości, jeśli obraz jest kolorowy
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image
    
    # Normalizacja wartości pikseli
    if gray.dtype != np.float64:
        gray = util.img_as_float(gray)
    
    # Parametry filtrów Gabora
    orientations = np.arange(num_orientations) * np.pi / num_orientations
    scales = [6, 12][:num_scales]  # Długości fal
    
    features = {}
    
    # Dla każdej skali i orientacji
    for i, lambd in enumerate(scales):
        for j, theta in enumerate(orientations):
            # Tworzymy filtr Gabora
            kernel = gabor_kernel(sigma=2.0, theta=theta, lambd=lambd, gamma=0.5, psi=0)
            
            # Filtrujemy obraz
            filtered_real = ndi.convolve(gray, kernel, mode='wrap')
            
            # Wyodrębniamy cechy z odpowiedzi filtru
            features[f'gabor_mean_s{i}_o{j}'] = np.mean(filtered_real)
            features[f'gabor_var_s{i}_o{j}'] = np.var(filtered_real)
            features[f'gabor_energy_s{i}_o{j}'] = np.sum(filtered_real**2)
            # Można także dodać maksimum i minimów odpowiedzi filtru
            features[f'gabor_max_s{i}_o{j}'] = np.max(filtered_real)
            features[f'gabor_min_s{i}_o{j}'] = np.min(filtered_real)
    
    return features

Poniższa funkcja ekstrahuje cechy obrazów Haralicka i Gabora i umieszcza w ramce danych

In [ ]:
def extract_features_from_image_array(images):
    all_features = []
    
    for i, img in enumerate(images):
        # Ekstrakcja cech
        features = {}
        features['image_index'] = i  # Zamiast ścieżki używamy indeksu
            
        # Cechy Haralicka
        haralick_features = extract_haralick_features(img)
        features.update(haralick_features)
            
        # Cechy Gabora
        gabor_features = extract_gabor_features(img)
        features.update(gabor_features)

        if (i+1) % 10 == 0:
                print(f"Przetworzono {i+1}/{len(images)} obrazów")
            
        all_features.append(features)
            
    
    features_df = pd.DataFrame(all_features)
    
    return features_df

In [ ]:
features_df = extract_features_from_image_array(images_cropped)

In [ ]:
features_df.head()

In [ ]:
features_df.to_csv("features_haralick_gabor_df.csv")

In [ ]:
df = pd.concat([df, features_df], axis=1)

In [ ]:
df.head(5)

In [ ]:
df.to_csv("df.csv")

In [ ]:
df.columns

## Embedding

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("/kaggle/input/openaiclip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("/kaggle/input/openaiclip-vit-base-patch32")

In [ ]:
def get_clip_embedding(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    return image_features.cpu().numpy().flatten()


In [ ]:
embedding_list = []

for img in images:
    embedding = get_clip_embedding(img)
    embedding_list.append(embedding)

import pandas as pd
embedding_df = pd.DataFrame(embedding_list)


In [ ]:
embedding_df.head()

In [ ]:
#embedding_df.to_csv("embedding_df.csv")

In [ ]:
df = pd.read_csv("/kaggle/input/df-features/df.csv")

In [ ]:
df = df.drop(columns = ['Unnamed: 0','Obraz'])

In [ ]:
embedding_df = pd.read_csv("/kaggle/input/embedding-df/embedding_df.csv")

In [ ]:
embedding_df = embedding_df.drop(columns = ['Unnamed: 0'])

In [ ]:
all_mixed_df = pd.concat([df, embedding_df], axis=1)

In [ ]:
scaler = StandardScaler()
embedding_df_scaled = scaler.fit_transform(embedding_df)
df_scaled = scaler.fit_transform(df)
all_mixed_df_scaled = scaler.fit_transform(all_mixed_df)

# Rzutowanie 

In [ ]:
#PCA
pca = PCA(n_components=2)
embedding_pca = pca.fit_transform(embedding_df_scaled)

plt.scatter(embedding_pca[:, 0], embedding_pca[:, 1])
plt.title("PCA: CLIP Embedding Clusters")
plt.show()

pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

plt.scatter(df_pca[:, 0], df_pca[:, 1])
plt.title("PCA: Df Clusters")
plt.show()

pca = PCA(n_components=2)
all_mixed_df_pca = pca.fit_transform(all_mixed_df_scaled)

plt.scatter(all_mixed_df_pca[:, 0], df_pca[:, 1])
plt.title("PCA: Df Clusters")
plt.show()

In [ ]:
#TSNE
tsne = TSNE(n_components=2, random_state=42)
embedding_tsne = tsne.fit_transform(embedding_df_scaled)

plt.scatter(embedding_tsne[:, 0], embedding_tsne[:, 1])
plt.title('t-SNE: Embedding ')
plt.show()

#---------
tsne = TSNE(n_components=2, random_state=42)
df_tsne = tsne.fit_transform(df_scaled)

plt.scatter(df_tsne[:, 0], df_tsne[:, 1])
plt.title('t-SNE: Df')
plt.show()

#--------
tsne = TSNE(n_components=2, random_state=42)
all_mixed_df_tsne = tsne.fit_transform(all_mixed_df_scaled)

plt.scatter(all_mixed_df_tsne[:, 0], all_mixed_df_tsne[:, 1])
plt.title('t-SNE: Embedding ')
plt.show()

In [ ]:
print(len(all_mixed_df_scaled[0]))
print(len(df_scaled[0]))
print(len(embedding_scaled[0]))

# Klasteryzacja

In [ ]:
#DBSCAN

dbscan = DBSCAN(eps=0.05, min_samples=5, metric = 'cosine')
labels = dbscan.fit_predict(df_tsne)

plt.scatter(df_tsne[:, 0], df_tsne[:, 1], c=labels, cmap='plasma')
plt.title("DBSCAN Clustering")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()


# Wszystko poniżej tego komentarza to syf


### K-means

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)  # dobierz n_clusters do liczby typów pistacji lub na podstawie elbow
labels = kmeans.fit_predict(embedding_scaled)

### DBSCAN

In [ ]:
dbscan = DBSCAN(eps=1.5, min_samples=5)
labels = dbscan.fit_predict(embedding_scaled)

In [ ]:
# Należy uruchomić po uruchomieniu odpowiedniej komórki z k-means lub DBSCAn, nie wszystko jak leci, bo zawsze DBSCAN będzie
embedding_df["cluster"] = labels

### Klasteryzacja oparta tylko na ramce df

K-means

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)  # dobierz n_clusters do liczby typów pistacji lub na podstawie elbow
labels = kmeans.fit_predict(df_scaled)

DBSCAN

In [ ]:
# Należy uruchomić po uruchomieniu odpowiedniej komórki z k-means lub DBSCAn, nie wszystko jak leci, bo zawsze DBSCAN będzie
df["cluster"] = labels

In [ ]:
dbscan = DBSCAN(eps=1.5, min_samples=5)
labels = dbscan.fit_predict(embedding_scaled)